In [1]:
!pip install scikit-learn torch pandas numpy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
df = pd.read_csv('./code_awgn_dataset.csv')

In [3]:
num_columns = len(df.columns)
num_columns

577

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from typing import List, Tuple
class ErrorCorrectionDataset:
    """
    Dataset for AWGN error correction.
    Each sample contains:
      - X: corrupted bytes as a sequence [seq_len, 1], normalized to [0, 1]
      - y: original bytes as normalized values [seq_len]
    """
    def __init__(self, dataframe, snr_db_pools: List[float], train_size: float = 0.8, seq_len=288, normalize=True):
        self.df = dataframe
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.snr_db_pools = snr_db_pools
        self.train_size = train_size
        self.seq_len = seq_len
        self.normalize = normalize
        self.corrupted_cols = [f"Corrupted_{i}" for i in range(seq_len)]
        self.original_cols = [f"Original_{i}" for i in range(seq_len)]
        self.X = dataframe[[f"Corrupted_{i}" for i in range(seq_len)]].values.astype(np.float32)
        self.Y = dataframe[[f"Original_{i}" for i in range(seq_len)]].values.astype(np.float32)
        self.min_val = -32768
        self.max_val = 32767
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        corrupted = self.X[idx]  # shape (288,)
        original = self.Y[idx]   # shape (288,)
        if self.normalize:
            corrupted = (corrupted - self.min_val) / (self.max_val - self.min_val)
            original = (original - self.min_val) / (self.max_val - self.min_val)
        X = torch.tensor(corrupted, dtype=torch.float32).unsqueeze(-1)  # [288, 1]
        y = torch.tensor(original, dtype=torch.float32)                 # [288]
        return X, y
    def _split_tensor(self, sub_df: pd.DataFrame) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """Helper to split a dataframe into train/test torch tensors."""
        n_train = int(self.train_size * len(sub_df))
        sub_df = sub_df.sample(frac=1, random_state=None).reset_index(drop=True)
        X = sub_df[self.corrupted_cols].values.astype(np.float32)
        y = sub_df[self.original_cols].values.astype(np.float32)
        min_val = -32768
        max_val = 32767
        X = (X - min_val) / (max_val - min_val)
        y = (y - min_val) / (max_val - min_val)
        X_train, X_test = X[:n_train], X[n_train:]
        y_train, y_test = y[:n_train], y[n_train:]
        return (
            torch.tensor(X_train, dtype=torch.float32).to(self.device),
            torch.tensor(y_train, dtype=torch.float32).to(self.device),
            torch.tensor(X_test, dtype=torch.float32).to(self.device),
            torch.tensor(y_test, dtype=torch.float32).to(self.device),
        )
    def prepare_datasets(self) -> List[Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]]:
        """
        Returns a list of datasets (X_train, y_train, X_test, y_test) for each SNR_DB pool.
        """
        datasets = []
        for snr_db in self.snr_db_pools:
            sub_df = self.df[self.df["SNR_DB"] == snr_db]
            datasets.append(self._split_tensor(sub_df))
        return datasets
    def generalized_dataset(self) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Returns a single dataset (X_train, y_train, X_test, y_test) combining all SNR_DB pools.
        """
        return self._split_tensor(self.df)

In [7]:
snr_db_pools = [5, 10, 20, 50, 100, 500, 1000]
snr_db_pools.reverse()
Dataset = ErrorCorrectionDataset(df, snr_db_pools, train_size=0.8, seq_len=288)

In [8]:
error_datasets = Dataset.prepare_datasets()
generalized_dataset = Dataset.generalized_dataset()

In [9]:
import torch
import torch.nn as nn

class AWGNErrorCorrector(nn.Module):
    def __init__(self, input_dim=288, d_model=256, nhead=8, num_layers=6, dim_feedforward=1024, dropout=0.1):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, d_model)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,
            activation="gelu"
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Output: predict 288 numbers per position
        self.output_fc = nn.Linear(d_model, input_dim)

    def forward(self, x):
        # x: [B, seq_len, 288]
        x = self.input_fc(x)               # -> [B, seq_len, d_model]
        x = self.transformer_encoder(x)    # -> [B, seq_len, d_model]
        x = self.output_fc(x)              # -> [B, seq_len, 288]
        return x


In [10]:
seq_len = 288
model = AWGNErrorCorrector().to(device)

criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
X_train_00 = error_datasets[0][0].unsqueeze(0)
y_train_00 = error_datasets[0][1].unsqueeze(0)

for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(X_train_00)  
    loss = criterion(outputs, y_train_00)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.724407
Epoch 2, Loss=0.488382
Epoch 3, Loss=0.260347
Epoch 4, Loss=0.168003
Epoch 5, Loss=0.132640
Epoch 6, Loss=0.120395
Epoch 7, Loss=0.120564
Epoch 8, Loss=0.124633
Epoch 9, Loss=0.127421
Epoch 10, Loss=0.127593
Epoch 11, Loss=0.125279
Epoch 12, Loss=0.120974
Epoch 13, Loss=0.115825
Epoch 14, Loss=0.111004
Epoch 15, Loss=0.107280
Epoch 16, Loss=0.105192
Epoch 17, Loss=0.104121
Epoch 18, Loss=0.103746
Epoch 19, Loss=0.103609
Epoch 20, Loss=0.103701
Epoch 21, Loss=0.103446
Epoch 22, Loss=0.102988
Epoch 23, Loss=0.102326
Epoch 24, Loss=0.101403
Epoch 25, Loss=0.100164
Epoch 26, Loss=0.099103
Epoch 27, Loss=0.097993
Epoch 28, Loss=0.097286
Epoch 29, Loss=0.096692
Epoch 30, Loss=0.096249
Epoch 31, Loss=0.096087
Epoch 32, Loss=0.096094
Epoch 33, Loss=0.095679
Epoch 34, Loss=0.095493
Epoch 35, Loss=0.095020
Epoch 36, Loss=0.094820
Epoch 37, Loss=0.094626
Epoch 38, Loss=0.094345
Epoch 39, Loss=0.094067
Epoch 40, Loss=0.093868
Epoch 41, Loss=0.093536
Epoch 42, Loss=0.093385
E

In [12]:
y_train_00.shape, X_train_00.shape

(torch.Size([1, 3968, 288]), torch.Size([1, 3968, 288]))

In [13]:
print("X_train_00 shape:", X_train_00.shape, X_train_00.dtype)
print("y_train_00 shape:", y_train_00.shape, y_train_00.dtype)

X_train_00 shape: torch.Size([1, 3968, 288]) torch.float32
y_train_00 shape: torch.Size([1, 3968, 288]) torch.float32


In [14]:
X_train_01 = error_datasets[1][0].unsqueeze(0)
y_train_01 = error_datasets[1][1].unsqueeze(0)

for epoch in range(80):
    optimizer.zero_grad()
    outputs = model(X_train_01)  
    loss = criterion(outputs, y_train_01)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.082670
Epoch 2, Loss=0.082664
Epoch 3, Loss=0.082659
Epoch 4, Loss=0.082654
Epoch 5, Loss=0.082664
Epoch 6, Loss=0.082662
Epoch 7, Loss=0.082657
Epoch 8, Loss=0.082667
Epoch 9, Loss=0.082662
Epoch 10, Loss=0.082665
Epoch 11, Loss=0.082668
Epoch 12, Loss=0.082666
Epoch 13, Loss=0.082661
Epoch 14, Loss=0.082663
Epoch 15, Loss=0.082659
Epoch 16, Loss=0.082653
Epoch 17, Loss=0.082662
Epoch 18, Loss=0.082659
Epoch 19, Loss=0.082658
Epoch 20, Loss=0.082658
Epoch 21, Loss=0.082667
Epoch 22, Loss=0.082656
Epoch 23, Loss=0.082654
Epoch 24, Loss=0.082654
Epoch 25, Loss=0.082655
Epoch 26, Loss=0.082656
Epoch 27, Loss=0.082657
Epoch 28, Loss=0.082660
Epoch 29, Loss=0.082664
Epoch 30, Loss=0.082649
Epoch 31, Loss=0.082655
Epoch 32, Loss=0.082653
Epoch 33, Loss=0.082651
Epoch 34, Loss=0.082649
Epoch 35, Loss=0.082654
Epoch 36, Loss=0.082657
Epoch 37, Loss=0.082659
Epoch 38, Loss=0.082654
Epoch 39, Loss=0.082637
Epoch 40, Loss=0.082649
Epoch 41, Loss=0.082652
Epoch 42, Loss=0.082649
E

In [15]:
X_train_02 = error_datasets[2][0].unsqueeze(0)
y_train_02 = error_datasets[2][1].unsqueeze(0)

for epoch in range(60):
    optimizer.zero_grad()
    outputs = model(X_train_02)  
    loss = criterion(outputs, y_train_02)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.082743
Epoch 2, Loss=0.082733
Epoch 3, Loss=0.082737
Epoch 4, Loss=0.082734
Epoch 5, Loss=0.082741
Epoch 6, Loss=0.082739
Epoch 7, Loss=0.082732
Epoch 8, Loss=0.082726
Epoch 9, Loss=0.082743
Epoch 10, Loss=0.082736
Epoch 11, Loss=0.082733
Epoch 12, Loss=0.082739
Epoch 13, Loss=0.082728
Epoch 14, Loss=0.082723
Epoch 15, Loss=0.082740
Epoch 16, Loss=0.082735
Epoch 17, Loss=0.082726
Epoch 18, Loss=0.082737
Epoch 19, Loss=0.082732
Epoch 20, Loss=0.082726
Epoch 21, Loss=0.082738
Epoch 22, Loss=0.082743
Epoch 23, Loss=0.082742
Epoch 24, Loss=0.082736
Epoch 25, Loss=0.082741
Epoch 26, Loss=0.082741
Epoch 27, Loss=0.082738
Epoch 28, Loss=0.082738
Epoch 29, Loss=0.082728
Epoch 30, Loss=0.082734
Epoch 31, Loss=0.082738
Epoch 32, Loss=0.082730
Epoch 33, Loss=0.082747
Epoch 34, Loss=0.082734
Epoch 35, Loss=0.082727
Epoch 36, Loss=0.082739
Epoch 37, Loss=0.082729
Epoch 38, Loss=0.082725
Epoch 39, Loss=0.082726
Epoch 40, Loss=0.082727
Epoch 41, Loss=0.082730
Epoch 42, Loss=0.082730
E

In [16]:
X_train_03 = error_datasets[3][0].unsqueeze(0)
y_train_03 = error_datasets[3][1].unsqueeze(0)

for epoch in range(40):
    optimizer.zero_grad()
    outputs = model(X_train_03)  
    loss = criterion(outputs, y_train_03)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.082597
Epoch 2, Loss=0.082590
Epoch 3, Loss=0.082591
Epoch 4, Loss=0.082583
Epoch 5, Loss=0.082595
Epoch 6, Loss=0.082587
Epoch 7, Loss=0.082594
Epoch 8, Loss=0.082589
Epoch 9, Loss=0.082589
Epoch 10, Loss=0.082595
Epoch 11, Loss=0.082597
Epoch 12, Loss=0.082591
Epoch 13, Loss=0.082590
Epoch 14, Loss=0.082592
Epoch 15, Loss=0.082584
Epoch 16, Loss=0.082595
Epoch 17, Loss=0.082586
Epoch 18, Loss=0.082585
Epoch 19, Loss=0.082585
Epoch 20, Loss=0.082595
Epoch 21, Loss=0.082587
Epoch 22, Loss=0.082590
Epoch 23, Loss=0.082599
Epoch 24, Loss=0.082586
Epoch 25, Loss=0.082588
Epoch 26, Loss=0.082585
Epoch 27, Loss=0.082578
Epoch 28, Loss=0.082584
Epoch 29, Loss=0.082587
Epoch 30, Loss=0.082581
Epoch 31, Loss=0.082591
Epoch 32, Loss=0.082584
Epoch 33, Loss=0.082593
Epoch 34, Loss=0.082587
Epoch 35, Loss=0.082589
Epoch 36, Loss=0.082589
Epoch 37, Loss=0.082581
Epoch 38, Loss=0.082585
Epoch 39, Loss=0.082589
Epoch 40, Loss=0.082580


In [17]:
X_train_04 = error_datasets[4][0].unsqueeze(0)
y_train_04 = error_datasets[4][1].unsqueeze(0)

for epoch in range(20):
    optimizer.zero_grad()
    outputs = model(X_train_04)  
    loss = criterion(outputs, y_train_04)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.082636
Epoch 2, Loss=0.082631
Epoch 3, Loss=0.082639
Epoch 4, Loss=0.082631
Epoch 5, Loss=0.082633
Epoch 6, Loss=0.082632
Epoch 7, Loss=0.082629
Epoch 8, Loss=0.082630
Epoch 9, Loss=0.082629
Epoch 10, Loss=0.082626
Epoch 11, Loss=0.082628
Epoch 12, Loss=0.082617
Epoch 13, Loss=0.082623
Epoch 14, Loss=0.082624
Epoch 15, Loss=0.082621
Epoch 16, Loss=0.082619
Epoch 17, Loss=0.082629
Epoch 18, Loss=0.082625
Epoch 19, Loss=0.082636
Epoch 20, Loss=0.082624


In [18]:
X_train_05 = error_datasets[5][0].unsqueeze(0)
y_train_05 = error_datasets[5][1].unsqueeze(0)

for epoch in range(15):
    optimizer.zero_grad()
    outputs = model(X_train_05)  
    loss = criterion(outputs, y_train_05)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.082686
Epoch 2, Loss=0.082677
Epoch 3, Loss=0.082697
Epoch 4, Loss=0.082694
Epoch 5, Loss=0.082688
Epoch 6, Loss=0.082686
Epoch 7, Loss=0.082683
Epoch 8, Loss=0.082684
Epoch 9, Loss=0.082681
Epoch 10, Loss=0.082679
Epoch 11, Loss=0.082681
Epoch 12, Loss=0.082687
Epoch 13, Loss=0.082687
Epoch 14, Loss=0.082685
Epoch 15, Loss=0.082680


In [19]:
X_train_06 = error_datasets[6][0].unsqueeze(0)
y_train_06 = error_datasets[6][1].unsqueeze(0)

for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_train_06)  
    loss = criterion(outputs, y_train_06)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.082733
Epoch 2, Loss=0.082718
Epoch 3, Loss=0.082715
Epoch 4, Loss=0.082719
Epoch 5, Loss=0.082729
Epoch 6, Loss=0.082734
Epoch 7, Loss=0.082727
Epoch 8, Loss=0.082726
Epoch 9, Loss=0.082725
Epoch 10, Loss=0.082719


In [ ]:
X_train = generalized_dataset[0].unsqueeze(0)
y_train = generalized_dataset[1].unsqueeze(0)

for epoch in range(20):
    optimizer.zero_grad()
    outputs = model(X_train)  
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.082671
Epoch 2, Loss=0.082662
Epoch 3, Loss=0.082663
Epoch 4, Loss=0.082664
Epoch 5, Loss=0.082665
Epoch 6, Loss=0.082665
Epoch 7, Loss=0.082660
Epoch 8, Loss=0.082658
Epoch 9, Loss=0.082658
Epoch 10, Loss=0.082655
Epoch 11, Loss=0.082656
Epoch 12, Loss=0.082657
Epoch 13, Loss=0.082655
Epoch 14, Loss=0.082655
Epoch 15, Loss=0.082654
Epoch 16, Loss=0.082653
Epoch 17, Loss=0.082655
Epoch 18, Loss=0.082653
Epoch 19, Loss=0.082650
Epoch 20, Loss=0.082653
Epoch 21, Loss=0.082653
Epoch 22, Loss=0.082651
Epoch 23, Loss=0.082651
Epoch 24, Loss=0.082651
Epoch 25, Loss=0.082650
Epoch 26, Loss=0.082650
Epoch 27, Loss=0.082649
Epoch 28, Loss=0.082646
Epoch 29, Loss=0.082648
Epoch 30, Loss=0.082649


In [ ]:
X_test = generalized_dataset[2].unsqueeze(0)
y_test = generalized_dataset[3].unsqueeze(0)

model.eval()
with torch.no_grad():
    # ---- Test metrics ----
    test_outputs = model(X_test)  
    test_loss = criterion(test_outputs, y_test)
    
    test_preds = test_outputs.argmax(dim=-1)
    test_targets = y_test
    test_acc = (test_preds == test_targets).float().mean().item()

    print(f"Test  Loss = {test_loss.item():.6f}, Test Byte Accuracy = {test_acc*100:.6f}%")

Test  Loss = 0.662494, Test Byte Accuracy = 0.500357%


In [ ]:
model.eval()
with torch.no_grad():
    # ---- Train metrics ----
    train_outputs = model(X_train)
    train_loss = criterion(
        train_outputs.view(-1, 256),     
        y_train.view(-1)        
    )

    train_preds = train_outputs.argmax(dim=-1)
    train_targets = y_train
    train_acc = (train_preds == train_targets).float().mean().item()

    print(f"Train Loss = {train_loss.item():.6f}, Train Byte Accuracy = {train_acc*100:.6f}%")

Train Loss = 0.662961, Train Byte Accuracy = 0.513877%


In [ ]:
import os

save_path = "./awgn_error_corrector.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

Model saved to ./error_correction/error_corrector_updated_dataset.pth
